In [ ]:
!pip install pytorch-tabnet
import pandas as pd
import numpy as np
from collections import Counter
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import StandardScaler
from pytorch_tabnet.pretraining import TabNetPretrainer

from matplotlib import pyplot as plt
%matplotlib inline

In [7]:
data_df = pd.read_csv('neo.csv')

In [ ]:
useless = ['id','name','orbiting_body']
data_df = data_df.drop(useless,axis=1)
data_df

In [ ]:
cat_cols = ['sentry_object','hazardous']
data_df[cat_cols] = data_df[cat_cols].astype(int)
data_df

In [ ]:
lencoder = LabelEncoder()
y = pd.DataFrame(lencoder.fit_transform(data_df['hazardous']), columns=['hazardous'])
y = y.to_numpy()
y

In [12]:
X = pd.DataFrame(data_df.drop("hazardous", axis = 1))
X

,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,sentry_object,absolute_magnitude
0,1.198271,2.679415,13569.249224,5.483974e+07,0,16.73
1,0.265800,0.594347,73588.726663,6.143813e+07,0,20.00
2,0.722030,1.614507,114258.692129,4.979872e+07,0,17.83
3,0.096506,0.215794,24764.303138,2.543497e+07,0,22.20
4,0.255009,0.570217,42737.733765,4.627557e+07,0,20.09
...,...,...,...,...,...,...
90831,0.026580,0.059435,52078.886692,1.230039e+07,0,25.00
90832,0.016771,0.037501,46114.605073,5.432121e+07,0,26.00
90833,0.031956,0.071456,7566.807732,2.840077e+07,0,24.60
90834,0.007321,0.016370,69199.154484,6.869206e+07,0,27.80


In [13]:
n_samples , n_features = X.shape

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

In [15]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [16]:
# TabNetPretrainer
unsupervised_model = TabNetPretrainer(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type='entmax',
)

Device used : cpu


In [17]:
import os
max_epochs = 20 if not os.getenv("CI", False) else 2

In [18]:
unsupervised_model.fit(
    X_train=X_train,
    eval_set=[X_test],
    max_epochs=max_epochs , patience=5,
    batch_size=1000, virtual_batch_size=128,
    num_workers=0,
    drop_last=False,
    pretraining_ratio=0.8,
) 

epoch 0  | loss: 12543587.76318| val_0_unsup_loss: 96318.02344|  0:00:04s
epoch 1  | loss: 233223.10346| val_0_unsup_loss: 144158.25|  0:00:08s
epoch 2  | loss: 125161.21953| val_0_unsup_loss: 64991.92969|  0:00:11s
epoch 3  | loss: 86817.67839| val_0_unsup_loss: 70440.13281|  0:00:15s
epoch 4  | loss: 69272.38803| val_0_unsup_loss: 38503.15234|  0:00:19s
epoch 5  | loss: 55371.6125| val_0_unsup_loss: 44683.96484|  0:00:23s
epoch 6  | loss: 50808.17507| val_0_unsup_loss: 18726.95703|  0:00:27s
epoch 7  | loss: 43053.10984| val_0_unsup_loss: 33225.86719|  0:00:31s
epoch 8  | loss: 41856.70332| val_0_unsup_loss: 18264.55664|  0:00:35s
epoch 9  | loss: 32090.00062| val_0_unsup_loss: 30265.56055|  0:00:38s
epoch 10 | loss: 29986.45319| val_0_unsup_loss: 43821.43359|  0:00:42s
epoch 11 | loss: 48308.87714| val_0_unsup_loss: 36138.57031|  0:00:46s
epoch 12 | loss: 44975.6021| val_0_unsup_loss: 21785.1582|  0:00:50s
epoch 13 | loss: 25001.36306| val_0_unsup_loss: 3817.89697|  0:00:54s
epoch 1

In [19]:
# Make reconstruction from a dataset
reconstructed_X = unsupervised_model.predict(X_test)

In [20]:
unsupervised_explain_matrix, unsupervised_masks = unsupervised_model.explain(X_test)

In [21]:
unsupervised_model.save_model('./test_pretrain')
loaded_pretrain = TabNetPretrainer()
loaded_pretrain.load_model('./test_pretrain.zip')

Successfully saved model at ./test_pretrain.zip
Device used : cpu
Device used : cpu


In [22]:
clf = TabNetClassifier(optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=2e-2),
                       scheduler_params={"step_size":10, # how to use learning rate scheduler
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='sparsemax' # This will be overwritten if using pretrain model
                      )

Device used : cpu


In [23]:
clf = TabNetClassifier()

Device used : cpu


In [24]:
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

In [25]:
clf.fit(
    X_train = X_train,
    y_train = y_train,
    eval_set=[(X_train,y_train),(X_test,y_test)],
    eval_name=['train', 'test'],
    eval_metric=['auc'],
    max_epochs=max_epochs , patience=20,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False,
    from_unsupervised=loaded_pretrain
)

/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/abstract_model.py:97: UserWarning: Pretraining: mask_type changed from sparsemax to entmax
  warnings.warn(wrn_msg)


Loading weights from unsupervised pretraining
epoch 0  | loss: 0.39554 | train_auc: 0.84741 | test_auc: 0.84772 |  0:00:04s
epoch 1  | loss: 0.32302 | train_auc: 0.88693 | test_auc: 0.8883  |  0:00:09s
epoch 2  | loss: 0.3165  | train_auc: 0.90318 | test_auc: 0.90353 |  0:00:13s
epoch 3  | loss: 0.3158  | train_auc: 0.9096  | test_auc: 0.90804 |  0:00:17s
epoch 4  | loss: 0.31065 | train_auc: 0.90947 | test_auc: 0.90878 |  0:00:22s
epoch 5  | loss: 0.30699 | train_auc: 0.91181 | test_auc: 0.91145 |  0:00:26s
epoch 6  | loss: 0.31087 | train_auc: 0.91201 | test_auc: 0.91082 |  0:00:31s
epoch 7  | loss: 0.3039  | train_auc: 0.91218 | test_auc: 0.91158 |  0:00:35s
epoch 8  | loss: 0.30633 | train_auc: 0.91146 | test_auc: 0.90921 |  0:00:40s
epoch 9  | loss: 0.30694 | train_auc: 0.91211 | test_auc: 0.91081 |  0:00:44s
epoch 10 | loss: 0.30534 | train_auc: 0.91253 | test_auc: 0.90892 |  0:00:48s
epoch 11 | loss: 0.30584 | train_auc: 0.91205 | test_auc: 0.91187 |  0:00:53s
epoch 12 | loss: 0

In [26]:
preds_valid = clf.predict_proba(X_test)
valid_auc = roc_auc_score(y_score=preds_valid[:,1], y_true=y_test)

In [27]:
valid_auc*100

91.18653404247158